In [ ]:
import pandas as pd
import json

with open('service_dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data['service'])

df

,contentTypeId,cat1,cat2,cat3,cat1Nm,cat2Nm,cat3Nm
0,12,A01,A0101,A01010100,자연,자연관광지,국립공원
1,12,A01,A0101,A01010200,자연,자연관광지,도립공원
2,12,A01,A0101,A01010300,자연,자연관광지,군립공원
3,12,A01,A0101,A01010400,자연,자연관광지,산
4,12,A01,A0101,A01010500,자연,자연관광지,자연생태관광지
...,...,...,...,...,...,...,...
62,14,A02,A0206,A02061000,인문(문화/예술/역사),문화시설,대형서점
63,14,A02,A0206,A02061100,인문(문화/예술/역사),문화시설,문화전수시설
64,14,A02,A0206,A02061200,인문(문화/예술/역사),문화시설,영화관
65,14,A02,A0206,A02061300,인문(문화/예술/역사),문화시설,어학당


In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import faiss
import numpy as np
import pandas as pd

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY를 .env 파일에 설정해주세요.")

client = OpenAI(api_key=openai_api_key)

# 예시 df (실제 데이터로 교체)
df = pd.DataFrame({
    "cat1Nm": ["자연", "인문(문화/예술/역사)", "문화시설"],
    "cat2Nm": ["자연관광지", "역사관광지", "문화시설"],
    "cat3Nm": ["국립공원", "고궁", "박물관"]
})

def get_openai_embedding(text):
    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response.data[0].embedding

texts = (df['cat1Nm'] + " " + df['cat2Nm'] + " " + df['cat3Nm']).tolist()
embeddings = [get_openai_embedding(t) for t in texts]
embeddings = np.array(embeddings).astype('float32')

d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

print(f"FAISS index에 {index.ntotal}개의 벡터 추가됨")

query = "국립공원"
query_emb = np.array([get_openai_embedding(query)]).astype('float32')
D, I = index.search(query_emb, k=3)

print("검색 결과 인덱스:", I)
print("검색 결과 거리:", D)
print("검색된 항목:")
print(df.iloc[I[0]])


APIRemovedInV1: 

You tried to access openai.Embedding, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742
